# QLoRA Fine-Tuning for LLM Unlearning

This notebook demonstrates how to fine-tune a large language model using QLoRA (Quantized Low-Rank Adaptation) for sentiment analysis tasks with unlearning capabilities.

## Setup Steps:
1. Install required dependencies
2. Setup utility functions
3. Configure model and training parameters
4. Load and prepare dataset
5. Train the model
6. Save the fine-tuned model

## 1. Install Dependencies

Install all required packages for QLoRA training.

In [24]:
!pip install -q datasets evaluate peft scikit-learn
!pip install -q torch torchinfo transformers trl
!pip install -q bitsandbytes wandb
!pip install -q accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

## 2. Import Required Libraries

In [25]:
import os
import sys
import time
import pickle
import datetime
import random
from copy import deepcopy
from argparse import Namespace

import numpy as np
from tqdm import tqdm
import evaluate

import torch
from torch.utils.data import DataLoader
from torchinfo import summary

from datasets import load_dataset, concatenate_datasets
from peft import get_peft_model, LoraConfig, TaskType, PeftConfig, PeftModel
from transformers import AutoTokenizer, TrainerState, TrainerControl, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, TrainerCallback, Trainer
from trl import SFTTrainer, SFTConfig

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

All libraries imported successfully!
PyTorch version: 2.3.0+cu121
CUDA available: True


## 3. Define Utility Functions

These functions are adapted from `utils.py`.

In [26]:
def get_data_path(dataset):
    """Get the HuggingFace dataset path based on dataset name."""
    if dataset.lower() == "sst2":
        data_path = "karuna-bhaila/Unlearning_SST2v3"
    elif dataset.lower() == 'yelp':
        data_path = "karuna-bhaila/Unlearning_Yelp_Polarity"
    else:
        # define dataset with the following splits:
        # train_retain, train_forget, test_retain, test_forget
        raise NotImplementedError(f"Dataset {dataset} not supported")

    return data_path


def preprocess_logits_for_metrics(logits, labels):
    """Preprocess logits for metric computation."""
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    # argmax to get the token ids
    return logits.argmax(dim=-1)


def compute_metrics(eval_pred):
    """Compute evaluation metrics."""
    f1_metric = evaluate.load("f1")
    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load('precision')
    recall_metric = evaluate.load('recall')

    logits, labels = eval_pred

    predictions = logits[:, :-1]
    labels = labels[:, 1:]

    check_labels = labels != -100

    last_token_predictions = []
    last_token_labels = []

    for idx in range(len(predictions)):
        last_token_predictions.append(predictions[idx][check_labels[idx]])
        last_token_labels.append(labels[idx][check_labels[idx]])

    f1 = f1_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='weighted')["f1"]
    accuracy = accuracy_metric.compute(predictions=last_token_predictions, references=last_token_labels)["accuracy"]
    precision = precision_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='micro')['precision']
    recall = recall_metric.compute(predictions=last_token_predictions, references=last_token_labels, average='micro')['recall']

    return {"f1-score": f1, 'accuracy': accuracy, 'precision': precision, 'recall': recall}


class CustomCallback(TrainerCallback):
    """Custom callback for additional evaluation during training."""
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.eval_dataset['train_retain'],
                                   metric_key_prefix="eval_train_retrain")
            self._trainer.evaluate(eval_dataset=self._trainer.eval_dataset['train_forget'],
                                   metric_key_prefix="eval_train_forget")
            return control_copy

print("Utility functions defined successfully!")

Utility functions defined successfully!


## 4. Define Model and Dataset Functions

These functions handle model initialization and dataset preparation.

In [27]:
def get_lora_model(model_checkpoints, rank=4, alpha=16, lora_dropout=0.1, bias='none'):
    """Initialize model with QLoRA configuration."""
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_checkpoints,
        device_map="auto",
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Configure LoRA with proper target modules and NO embedding/head training
    if model_checkpoints == 'mistralai/Mistral-7B-v0.1' or model_checkpoints == 'meta-llama/Llama-2-7b-hf':
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=rank,
            lora_alpha=alpha,
            lora_dropout=lora_dropout,
            bias=bias,
            target_modules=[
                "q_proj",
                "v_proj",
            ],
            inference_mode=False,  # Training mode
            modules_to_save=None,  # Don't train embeddings or LM head
        )
    else:
        # For OPT models, explicitly target attention layers only
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=rank,
            lora_alpha=alpha,
            lora_dropout=lora_dropout,
            bias=bias,
            target_modules=["q_proj", "v_proj", "k_proj", "out_proj"],  # Explicit target modules for OPT
            inference_mode=False,  # Training mode
            modules_to_save=None,  # Don't train embeddings or LM head
        )

    return model, tokenizer, peft_config


def get_unlearn_dataset(
        data_path,
        tokenizer,
        add_prefix_space=True,
        max_length=1024,
        truncation=True
):
    """Load and prepare the unlearning dataset."""
    # Format: prompt and completion for completion-only loss
    # The model will only compute loss on the completion part
    def _preprocessing_sentiment(examples):
        prompt = f"""### Text: {examples['text']}\n\n### Question: What is the sentiment of the given text?\n\n### Sentiment:"""
        completion = f""" {examples['label_text']}"""
        # Combine for SFTTrainer with completion_only_loss
        return {"text": prompt + completion}

    data = load_dataset(data_path)

    data = data.map(_preprocessing_sentiment, batched=False)
    data = data.remove_columns(['label', 'label_text'])
    data.set_format("torch")

    print(data)

    return data

print("Model and dataset functions defined successfully!")

Model and dataset functions defined successfully!


## 5. Login to Weights & Biases (Optional)

If you want to track your training with W&B, login here. Otherwise, you can skip this step or disable W&B in training arguments.

In [28]:
import wandb
import os
# Login to W&B (you'll be prompted for your API key)
# wandb.login()

# Or disable W&B tracking
os.environ["WANDB_DISABLED"] = "true"

print("W&B configuration set!")

W&B configuration set!


## 6. Configure Training Parameters

Modify these parameters according to your needs.

In [29]:
# Create arguments namespace (similar to command-line arguments)
args = Namespace(
    # Dataset configuration
    dataset="sst2",  # Options: 'sst2', 'yelp'

    # Model configuration
    # model_name="meta-llama/Llama-2-7b-hf",  # Model checkpoint path
    model_name="facebook/opt-1.3b",  # Model checkpoint path
    # Alternative models:
    # - "facebook/opt-1.3b"
    # - "meta-llama/Llama-2-13b-hf"
    # - "mistralai/Mistral-7B-v0.1"

    # Output configuration
    output_path=None,  # Will be auto-generated if None

    # Training hyperparameters
    max_length=1024,
    lr=1e-4,
    train_batch_size=45,  # Adjust based on your GPU memory
    eval_batch_size=45,
    num_epochs=5,
    weight_decay=0.001,

    # LoRA configuration
    lora_rank=16,
    lora_alpha=64,
    lora_dropout=0.1,
    lora_bias='none',  # Options: 'lora_only', 'none', 'all'

    # Model-specific settings
    set_pad_id=False,  # Set to True for Mistral-7B
)

print("Configuration:")
print(f"  Dataset: {args.dataset}")
print(f"  Model: {args.model_name}")
print(f"  Learning Rate: {args.lr}")
print(f"  Batch Size: {args.train_batch_size}")
print(f"  Epochs: {args.num_epochs}")
print(f"  LoRA Rank: {args.lora_rank}")
print(f"  LoRA Alpha: {args.lora_alpha}")

Configuration:
  Dataset: sst2
  Model: facebook/opt-1.3b
  Learning Rate: 0.0001
  Batch Size: 45
  Epochs: 5
  LoRA Rank: 16
  LoRA Alpha: 64


## 7. Setup Output Directory and W&B Project

In [30]:
# Determine model name for output directory
if 'llama-2-7b' in args.model_name.lower():
    model_name = 'llama-2-7b-hf'
elif 'llama-2-13b' in args.model_name.lower():
    model_name = 'llama-2-13b-hf'
elif 'opt-1.3b' in args.model_name.lower():
    model_name = 'opt-1.3b'
elif 'mistral' in args.model_name.lower():
    model_name = 'mistral-7b'
else:
    model_name = 'custom-model'

# Setup W&B project (if enabled)
if os.environ.get("WANDB_DISABLED") != "true":
    os.environ["WANDB_LOG_MODEL"] = "all"
    os.environ["WANDB_PROJECT"] = f'qlora_{model_name.lower()}_{args.dataset.lower()}'

# Get data path
data_path = get_data_path(args.dataset)

# Setup output directory
if args.output_path is None:
    args.output_path = f'qlora_checkpoints/{model_name.lower()}-qlora-{args.dataset.lower()}'

os.makedirs(args.output_path, exist_ok=True)

# Save arguments
with open(os.path.join(args.output_path, 'arguments.txt'), 'w') as f:
    for k, v in args.__dict__.items():
        f.write(f'{k}: {v}\n')

print(f"Output directory: {args.output_path}")
print(f"Data path: {data_path}")

Output directory: qlora_checkpoints/opt-1.3b-qlora-sst2
Data path: karuna-bhaila/Unlearning_SST2v3


## 8. Initialize Model and Tokenizer

In [ ]:
from huggingface_hub import login

# Login to Hugging Face Hub using the token stored in Colab Secrets
try:
    hf_token = 'ADD_HERE'
    if hf_token:
        login(hf_token)
        print("Successfully logged in to Hugging Face Hub.")
    else:
        print("Hugging Face token not found in Colab Secrets. Please add it.")
except Exception as e:
    print(f"An error occurred during Hugging Face login: {e}")

Successfully logged in to Hugging Face Hub.


In [32]:
print("Loading model and tokenizer...")
print("This may take a few minutes...")

model, tokenizer, lora_config = get_lora_model(
    args.model_name,
    rank=args.lora_rank,
    alpha=args.lora_alpha,
    lora_dropout=args.lora_dropout,
    bias=args.lora_bias
)

# CRITICAL: Explicitly freeze ALL base model parameters before applying LoRA
# This prevents SFTTrainer from making embeddings/layer norms trainable
print("\n🔒 Freezing base model parameters...")
for name, param in model.named_parameters():
    param.requires_grad = False  # Freeze everything first

print("✅ All base model parameters frozen!")

# Now apply LoRA using get_peft_model - this will ONLY make LoRA params trainable
model = get_peft_model(model, lora_config)

print("✅ LoRA adapters added!")
print("\nModel loaded successfully!")
print("\nModel Summary:")
summary(model)

Loading model and tokenizer...
This may take a few minutes...

🔒 Freezing base model parameters...
✅ All base model parameters frozen!
✅ LoRA adapters added!

Model loaded successfully!

Model Summary:


Layer (type:depth-idx)                                                 Param #
PeftModelForCausalLM                                                   --
├─LoraModel: 1-1                                                       --
│    └─OPTForCausalLM: 2-1                                             --
│    │    └─OPTModel: 3-1                                              718,069,760
│    │    └─Linear: 3-2                                                (102,957,056)
Total params: 821,026,816
Trainable params: 6,291,456
Non-trainable params: 814,735,360

## 8.7 Evaluate Base Model (Before Fine-tuning)

Let's evaluate the base model's accuracy before we fine-tune it. This gives us a baseline to compare against.

In [33]:
# Ensure the dataset is loaded before running zero-shot evaluation
if 'dataset' not in globals():
    print("Loading dataset before evaluation...")
    dataset = get_unlearn_dataset(
        data_path,
        tokenizer=tokenizer,
        max_length=args.max_length,
        add_prefix_space=True,
        truncation=True,
    )
    print("\nDataset splits:")
    for split in dataset.keys():
        print(f"  {split}: {len(dataset[split])} examples")
else:
    print("Dataset already loaded.")

Dataset already loaded.


In [23]:
print("Evaluating base model before fine-tuning...")
print("This will give us a baseline to compare against.")

# For base model evaluation, we'll use a simpler approach
# that manually computes accuracy without the complex metrics
from torch.utils.data import DataLoader
from tqdm import tqdm

def evaluate_base_model_simple(model, dataset, tokenizer, batch_size=8):
    """
    Simple evaluation that computes accuracy by checking if the model
    generates the correct sentiment token.
    """
    model.eval()

    correct = 0
    total = 0

    # Map sentiment labels to expected tokens
    sentiment_mapping = {
        'positive': tokenizer.encode(' positive', add_special_tokens=False)[0],
        'negative': tokenizer.encode(' negative', add_special_tokens=False)[0],
        'Positive': tokenizer.encode(' Positive', add_special_tokens=False)[0],
        'Negative': tokenizer.encode(' Negative', add_special_tokens=False)[0],
    }

    with torch.no_grad():
        for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluating"):
            batch_end = min(i + batch_size, len(dataset))
            batch = dataset[i:batch_end]

            for example in batch['text']:
                # Split into prompt and completion
                if "### Sentiment:" in example:
                    prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
                    true_sentiment = example.split("### Sentiment:")[1].strip()

                    # Tokenize prompt
                    inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {k: v.to(model.device) for k, v in inputs.items()}

                    # Generate next token
                    outputs = model(**inputs)
                    logits = outputs.logits
                    next_token_logits = logits[0, -1, :]
                    predicted_token = torch.argmax(next_token_logits).item()

                    # Check if predicted token matches any sentiment token
                    true_sentiment_lower = true_sentiment.lower().strip()

                    # Get expected token(s)
                    expected_tokens = []
                    if 'positive' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('positive'), sentiment_mapping.get('Positive')]
                    elif 'negative' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('negative'), sentiment_mapping.get('Negative')]

                    expected_tokens = [t for t in expected_tokens if t is not None]

                    if predicted_token in expected_tokens:
                        correct += 1

                    total += 1

    accuracy = correct / total if total > 0 else 0.0
    return accuracy

print("\n📊 Base Model Evaluation Results:")
print("="*60)

# Evaluate on all 4 splits: train_retain, train_forget, test_retain, test_forget
print("\n🔹 Evaluating Train Retain (Base Model)...")
train_retain_acc = evaluate_base_model_simple(model, dataset['train_retain'], tokenizer, batch_size=8)
print(f"  Accuracy: {train_retain_acc:.4f}")

print("\n🔹 Evaluating Train Forget (Base Model)...")
train_forget_acc = evaluate_base_model_simple(model, dataset['train_forget'], tokenizer, batch_size=8)
print(f"  Accuracy: {train_forget_acc:.4f}")

print("\n🔹 Evaluating Test Retain (Base Model)...")
test_retain_acc = evaluate_base_model_simple(model, dataset['test_retain'], tokenizer, batch_size=8)
print(f"  Accuracy: {test_retain_acc:.4f}")

print("\n🔹 Evaluating Test Forget (Base Model)...")
test_forget_acc = evaluate_base_model_simple(model, dataset['test_forget'], tokenizer, batch_size=8)
print(f"  Accuracy: {test_forget_acc:.4f}")

print("="*60)

# Store baseline results for comparison later (4 splits)
baseline_results = {
    'train_retain': {'accuracy': train_retain_acc},
    'train_forget': {'accuracy': train_forget_acc},
    'test_retain': {'accuracy': test_retain_acc},
    'test_forget': {'accuracy': test_forget_acc}
}

print("\n✅ Base model evaluation complete!")
print(f"Note: Using simplified accuracy metric (correct sentiment token prediction)")
print(f"Note: Low accuracy (~1-5%) is expected for zero-shot on untuned base models")

Evaluating base model before fine-tuning...
This will give us a baseline to compare against.

📊 Base Model Evaluation Results:

🔹 Evaluating Train Retain (Base Model)...


Evaluating:   6%|▌         | 328/5792 [03:10<52:52,  1.72it/s]  


KeyboardInterrupt: 

In [34]:
# Debug zero-shot behavior: inspect tokens and a few example predictions
from collections import Counter
from tqdm import tqdm

def debug_zero_shot(model, dataset_split, tokenizer, n_samples=20):
    model.eval()
    # Show expected token ids for label variants
    variants = [' positive', 'negative', ' Positive', 'Negative', 'positive', ' negative']
    print("Label token variants and token ids/strings:")
    for v in variants:
        ids = tokenizer.encode(v, add_special_tokens=False)
        print(f"  variant={repr(v):10s} -> ids={ids} -> tokens={[tokenizer.convert_ids_to_tokens(i) for i in ids]} -> decoded={tokenizer.decode(ids)}")

    unknown_count = 0
    mapped_count = 0
    pred_label_counter = Counter()
    true_label_counter = Counter()

    for i, example in enumerate(dataset_split['text'][:n_samples]):
        if "### Sentiment:" not in example:
            print(f"[{i}] SKIP: no sentiment marker in example")
            continue

        prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
        true_sentiment = example.split("### Sentiment:")[1].strip()
        true_lab = 'positive' if 'positive' in true_sentiment.lower() else ('negative' if 'negative' in true_sentiment.lower() else 'unknown')
        true_label_counter[true_lab] += 1

        inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            out = model(**inputs)
            logits = out.logits  # (1, seq_len, vocab)
            pred_id = int(logits[0, -1, :].argmax().cpu().numpy())
        pred_token_str = tokenizer.decode([pred_id])
        # Map predicted token to label if it matches one of the single-token encodings we care about
        if pred_id in tokenizer.encode(' positive', add_special_tokens=False) or pred_token_str.strip().lower().startswith('positive'):
            pred_lab = 'positive'
            mapped_count += 1
        elif pred_id in tokenizer.encode(' negative', add_special_tokens=False) or pred_token_str.strip().lower().startswith('negative'):
            pred_lab = 'negative'
            mapped_count += 1
        else:
            pred_lab = 'unknown'
            unknown_count += 1

        pred_label_counter[pred_lab] += 1

        print(f"[{i}] True: {true_sentiment!r} (mapped={true_lab}) | pred_id={pred_id} | pred_token={pred_token_str!r} | pred_label={pred_lab}")

    total_seen = n_samples
    print("\\nSummary:")
    print(f"  Total examples inspected: {total_seen}")
    print(f"  Mapped predictions (positive/negative): {mapped_count}")
    print(f"  Unknown predictions: {unknown_count}")
    print("  Pred label counts:", dict(pred_label_counter))
    print("  True label counts (sample):", dict(true_label_counter))

# Usage example (run one split)
debug_zero_shot(model, dataset['test_retain'], tokenizer, n_samples=30)

Label token variants and token ids/strings:
  variant=' positive' -> ids=[1313] -> tokens=['Ġpositive'] -> decoded= positive
  variant='negative' -> ids=[33407] -> tokens=['negative'] -> decoded=negative
  variant=' Positive' -> ids=[25968] -> tokens=['ĠPositive'] -> decoded= Positive
  variant='Negative' -> ids=[37128, 3693] -> tokens=['Neg', 'ative'] -> decoded=Negative
  variant='positive' -> ids=[22173] -> tokens=['positive'] -> decoded=positive
  variant=' negative' -> ids=[2430] -> tokens=['Ġnegative'] -> decoded= negative
[0] True: 'negative' (mapped=negative) | pred_id=147 | pred_token=' where' | pred_label=unknown
[1] True: 'negative' (mapped=negative) | pred_id=614 | pred_token=' low' | pred_label=unknown
[2] True: 'negative' (mapped=negative) | pred_id=20 | pred_token=' The' | pred_label=unknown
[3] True: 'negative' (mapped=negative) | pred_id=5 | pred_token=' the' | pred_label=unknown
[4] True: 'positive' (mapped=positive) | pred_id=50118 | pred_token='\n' | pred_label=unkn

## 8.5 Verify LoRA Configuration

**IMPORTANT:** Check that only LoRA adapters are trainable, not the full model!

In [35]:
# Verify LoRA parameters - should be ~1-4M trainable params, not 200M+!
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    trainable_percentage = 100 * trainable_params / all_param

    print(f"╔═══════════════════════════════════════════════╗")
    print(f"║          LoRA Parameter Verification         ║")
    print(f"╠═══════════════════════════════════════════════╣")
    print(f"║ Total params:      {all_param:>15,} ║")
    print(f"║ Trainable params:  {trainable_params:>15,} ║")
    print(f"║ Non-trainable:     {all_param - trainable_params:>15,} ║")
    print(f"║ Trainable %:       {trainable_percentage:>14.4f}% ║")
    print(f"╚═══════════════════════════════════════════════╝")

    # Validation checks
    if trainable_percentage > 5.0:
        print("\n⚠️  WARNING: Trainable params > 5%! This is NOT typical for LoRA!")
        print("   Expected: 0.1-1% for LoRA (rank 8-64)")
        print("   Check that embeddings/LM head are NOT being trained.")
    elif trainable_percentage < 0.05:
        print("\n⚠️  WARNING: Trainable params < 0.05%! This might be too few.")
        print("   Consider increasing LoRA rank or checking target_modules.")
    else:
        print("\n✅ Parameter count looks good for LoRA training!")

    return trainable_params, all_param

# Print detailed parameter info
print("\n" + "="*60)
trainable, total = print_trainable_parameters(model)
print("="*60)

# Print which modules are trainable
print("\n📋 Trainable modules:")
trainable_modules = []
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_modules.append(name)

if len(trainable_modules) <= 50:  # Only print if reasonable number
    for name in trainable_modules:
        print(f"  ✓ {name}")
else:
    print(f"  Total: {len(trainable_modules)} trainable modules")
    print("  Sample (first 10):")
    for name in trainable_modules[:10]:
        print(f"  ✓ {name}")


╔═══════════════════════════════════════════════╗
║          LoRA Parameter Verification         ║
╠═══════════════════════════════════════════════╣
║ Total params:          718,069,760 ║
║ Trainable params:        6,291,456 ║
║ Non-trainable:         711,778,304 ║
║ Trainable %:               0.8762% ║
╚═══════════════════════════════════════════════╝

✅ Parameter count looks good for LoRA training!

📋 Trainable modules:
  Total: 192 trainable modules
  Sample (first 10):
  ✓ base_model.model.model.decoder.layers.0.self_attn.k_proj.lora_A.default.weight
  ✓ base_model.model.model.decoder.layers.0.self_attn.k_proj.lora_B.default.weight
  ✓ base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight
  ✓ base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight
  ✓ base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight
  ✓ base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight
  ✓ base_model.model.mo

### Expected LoRA Parameter Counts

For reference, here are typical trainable parameter counts for LoRA:

| Model | Total Params | LoRA Rank | Expected Trainable | % of Total |
|-------|-------------|-----------|-------------------|------------|
| OPT-1.3B | ~1.3B | 8 | ~2M | 0.15% |
| OPT-1.3B | **16** | **~4M** | **0.3%** |
| OPT-1.3B | 32 | ~8M | 0.6% |
| Llama-2-7B | ~7B | 16 | ~20M | 0.3% |
| Llama-2-7B | 64 | ~80M | 1.1% |

**🎯 Your current settings (rank=16):**
- Should have ~4M trainable params (0.3%)
- If you see >100M, embeddings/LM head are incorrectly trainable!

## 9. Load and Prepare Dataset

In [36]:
print("Loading dataset...")

dataset = get_unlearn_dataset(
    data_path,
    tokenizer=tokenizer,
    max_length=args.max_length,
    add_prefix_space=True,
    truncation=True,
)

print("\nDataset splits:")
for split in dataset.keys():
    print(f"  {split}: {len(dataset[split])} examples")

Loading dataset...
DatasetDict({
    train_forget: Dataset({
        features: ['text'],
        num_rows: 1425
    })
    test_forget: Dataset({
        features: ['text'],
        num_rows: 610
    })
    train_retain: Dataset({
        features: ['text'],
        num_rows: 46331
    })
    test_retain: Dataset({
        features: ['text'],
        num_rows: 19855
    })
})

Dataset splits:
  train_forget: 1425 examples
  test_forget: 610 examples
  train_retain: 46331 examples
  test_retain: 19855 examples


In [ ]:
# Save dataset splits locally in case using a different evaluation strategy later
print("Saving dataset splits locally...")

import os

# Create datasets directory in output path
datasets_dir = os.path.join(args.output_path, 'datasets')
os.makedirs(datasets_dir, exist_ok=True)

# Save each split
for split_name in dataset.keys():
    split_path = os.path.join(datasets_dir, f'{split_name}.pkl')
    dataset[split_name].save_to_disk(os.path.join(datasets_dir, split_name))
    print(f"  ✓ Saved {split_name}: {len(dataset[split_name])} examples")

print(f"\n✅ All dataset splits saved to: {datasets_dir}")
print("\nTo load later, use:")
print("  from datasets import load_from_disk")
print(f"  dataset = load_from_disk('{datasets_dir}/train_retain')")
print("\nSaved splits:")
for split_name in dataset.keys():
    print(f"  - {split_name}: {os.path.join(datasets_dir, split_name)}")

## 10. Configure Training Arguments

Using SFTConfig with completion_only_loss for modern TRL approach.

In [37]:
# Use SFTConfig instead of TrainingArguments for modern TRL
# This includes completion_only_loss=True to mask loss on prompt tokens
training_args = SFTConfig(
    output_dir=args.output_path,
    learning_rate=args.lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    num_train_epochs=args.num_epochs,
    weight_decay=args.weight_decay,
    eval_strategy="no",
    save_strategy="no",
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    fp16=True,
    report_to="wandb" if os.environ.get("WANDB_DISABLED") != "true" else "none",
    run_name=f'lr={args.lr}',
    max_grad_norm=0.3,
    metric_for_best_model="eval_test_loss",
    logging_steps=100,
    # SFT-specific parameters
    max_length=args.max_length,
    dataset_text_field='text',
    packing=False,  # Disable packing for clearer training
    # IMPORTANT: completion_only_loss masks loss on prompt tokens
    # Only the completion (after "### Sentiment:") is used for loss
    # This replaces the deprecated DataCollatorForCompletionOnlyLM
)

print("Training arguments configured with SFTConfig!")
print("Using completion_only_loss=True (modern TRL approach)")

Training arguments configured with SFTConfig!
Using completion_only_loss=True (modern TRL approach)


## 11. Set Padding Token (if needed)

In [38]:
if args.set_pad_id:
    model.config.pad_token_id = model.config.eos_token_id
    print("Padding token ID set to EOS token ID")

# Ensure model is on GPU
if model.device.type != 'cuda':
    model = model.to('cuda')
    print("Model moved to GPU")
else:
    print("Model already on GPU")

Model already on GPU


## 12. Initialize Trainer

In [39]:
print("Initializing trainer...")

# Since we already applied PEFT with get_peft_model, don't pass peft_config here
# This prevents SFTTrainer from re-applying PEFT and making extra params trainable
trainer = SFTTrainer(
    model=model,  # Already has LoRA adapters applied
    args=training_args,  # SFTConfig with completion_only_loss=True
    peft_config=lora_config,  # DON'T pass this - already applied! # This line was commented out, uncommenting it based on error
    processing_class=tokenizer,
    train_dataset=concatenate_datasets([dataset['train_retain'], dataset['train_forget']]),
    eval_dataset={"test": concatenate_datasets([dataset['test_retain'], dataset['test_forget']])},
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

# Add custom callback
trainer.add_callback(CustomCallback(trainer))

print("Trainer initialized successfully!")
print("Note: Using completion_only_loss - only sentiment label tokens contribute to loss")

Initializing trainer...


/home/tanish/miniconda3/envs/dl_proj/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/home/tanish/miniconda3/envs/dl_proj/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating test dataset: 100%|██████████| 20465/20465 [00:00<00:00, 885057.65 examples/s]


Trainer initialized successfully!
Note: Using completion_only_loss - only sentiment label tokens contribute to loss


## 13. Train the Model

This will start the training process. Depending on your model size and dataset, this may take several hours.

In [40]:
print("Starting training...")
print(f"Training for {args.num_epochs} epochs")
print("="*50)

start_time = time.perf_counter()

trainer.train()

runtime = time.perf_counter() - start_time

print("="*50)
print(f"Training completed!")
print(f"Total training time: {runtime:.2f} seconds ({runtime/60:.2f} minutes)")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Starting training...
Training for 5 epochs


/home/tanish/miniconda3/envs/dl_proj/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.422200
200,1.288700
300,1.281000
400,1.260800
500,1.246800
600,1.235800
700,1.225800
800,1.219900
900,1.206800
1000,1.217900


Training completed!
Total training time: 6101.90 seconds (101.70 minutes)


## 14. Save the Fine-tuned Model

In [41]:
# Save the fine-tuned model locally
print("Saving model locally...")

# Save to local output directory
final_model_path = os.path.join(args.output_path, 'final_model')
trainer.save_model(final_model_path)

# Save tokenizer
tokenizer.save_pretrained(final_model_path)

# Save arguments
with open(os.path.join(final_model_path, 'arguments.txt'), 'w') as f:
    for k, v in args.__dict__.items():
        f.write(f'{k}: {v}\n')

# Save evaluation results if they exist
import shutil
eval_comparison_path = os.path.join(args.output_path, 'evaluation_comparison.pkl')
if os.path.exists(eval_comparison_path):
    shutil.copy(eval_comparison_path, os.path.join(final_model_path, 'evaluation_comparison.pkl'))
    print("Evaluation comparison saved.")

print(f"\n✅ Model saved successfully!")
print(f"  Location: {final_model_path}")
print(f"\nModel files saved:")
print(f"  - LoRA adapters")
print(f"  - Tokenizer")
print(f"  - Training arguments")
print(f"  - Evaluation results (if available)")
print("\nYou can now use this model for inference!")

Saving model locally...

✅ Model saved successfully!
  Location: qlora_checkpoints/opt-1.3b-qlora-sst2/final_model

Model files saved:
  - LoRA adapters
  - Tokenizer
  - Training arguments
  - Evaluation results (if available)

You can now use this model for inference!

✅ Model saved successfully!
  Location: qlora_checkpoints/opt-1.3b-qlora-sst2/final_model

Model files saved:
  - LoRA adapters
  - Tokenizer
  - Training arguments
  - Evaluation results (if available)

You can now use this model for inference!


## 15. Evaluate the Model (Optional)

This cell evaluates the fine-tuned model on all 4 splits:
- **Train Retain** - training data that should be retained
- **Train Forget** - training data that should be "forgotten" (for unlearning experiments)
- **Test Retain** - held-out test data for retained knowledge
- **Test Forget** - held-out test data for forgotten knowledge

The comparison section shows the improvement (Δ) from the base model to the fine-tuned model for each split.

In [ ]:
# Evaluate the fine-tuned model on all 4 splits
# Using manual evaluation (not trainer.evaluate) to avoid SFT data processing issues

print("="*60)
print("📊 Fine-tuned Model Evaluation Results:")
print("="*60)

from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

def evaluate_finetuned_model(model, dataset, tokenizer, batch_size=8, split_name=""):
    """
    Evaluate fine-tuned model using the same approach as base model evaluation.
    Returns accuracy only (matching baseline format).
    """
    model.eval()
    
    correct = 0
    total = 0
    
    # Map sentiment labels to expected tokens
    sentiment_mapping = {
        'positive': tokenizer.encode(' positive', add_special_tokens=False)[0],
        'negative': tokenizer.encode(' negative', add_special_tokens=False)[0],
        'Positive': tokenizer.encode(' Positive', add_special_tokens=False)[0],
        'Negative': tokenizer.encode(' Negative', add_special_tokens=False)[0],
    }
    
    with torch.no_grad():
        for i in tqdm(range(0, len(dataset), batch_size), desc=f"Evaluating {split_name}"):
            batch_end = min(i + batch_size, len(dataset))
            batch = dataset[i:batch_end]
            
            for example in batch['text']:
                # Split into prompt and completion
                if "### Sentiment:" in example:
                    prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
                    true_sentiment = example.split("### Sentiment:")[1].strip()
                    
                    # Tokenize prompt
                    inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {k: v.to(model.device) for k, v in inputs.items()}
                    
                    # Get model prediction
                    outputs = model(**inputs)
                    logits = outputs.logits
                    next_token_logits = logits[0, -1, :]
                    predicted_token = torch.argmax(next_token_logits).item()
                    
                    # Check if predicted token matches any sentiment token
                    true_sentiment_lower = true_sentiment.lower().strip()
                    
                    # Get expected token(s)
                    expected_tokens = []
                    if 'positive' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('positive'), sentiment_mapping.get('Positive')]
                    elif 'negative' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('negative'), sentiment_mapping.get('Negative')]
                    
                    expected_tokens = [t for t in expected_tokens if t is not None]
                    
                    if predicted_token in expected_tokens:
                        correct += 1
                    
                    total += 1
    
    accuracy = correct / total if total > 0 else 0.0
    return {'accuracy': accuracy}

# Evaluate on all 4 splits
print("\n🔹 Evaluating Train Retain (Fine-tuned Model)...")
train_retain_finetuned = evaluate_finetuned_model(model, dataset['train_retain'], tokenizer, batch_size=8, split_name="Train Retain")
print(f"  Accuracy: {train_retain_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Train Forget (Fine-tuned Model)...")
train_forget_finetuned = evaluate_finetuned_model(model, dataset['train_forget'], tokenizer, batch_size=8, split_name="Train Forget")
print(f"  Accuracy: {train_forget_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Test Retain (Fine-tuned Model)...")
test_retain_finetuned = evaluate_finetuned_model(model, dataset['test_retain'], tokenizer, batch_size=8, split_name="Test Retain")
print(f"  Accuracy: {test_retain_finetuned['accuracy']:.4f}")

print("\n🔹 Evaluating Test Forget (Fine-tuned Model)...")
test_forget_finetuned = evaluate_finetuned_model(model, dataset['test_forget'], tokenizer, batch_size=8, split_name="Test Forget")
print(f"  Accuracy: {test_forget_finetuned['accuracy']:.4f}")






📊 Fine-tuned Model Evaluation Results:

🔹 Evaluating Train Retain (Fine-tuned Model)...


Evaluating Train Retain:   0%|          | 0/5792 [00:00<?, ?it/s]

Evaluating Train Retain: 100%|██████████| 5792/5792 [1:00:49<00:00,  1.59it/s]



  Accuracy: 0.9661

🔹 Evaluating Train Forget (Fine-tuned Model)...


Evaluating Train Forget: 100%|██████████| 179/179 [01:50<00:00,  1.63it/s]


...

  Accuracy: 0.9526

🔹 Evaluating Test Forget (Fine-tuned Model)...


Evaluating Test Forget: 100%|██████████| 77/77 [00:46<00:00,  1.66it/s]



  Accuracy: 0.9492

📈 Comparison: Base vs Fine-tuned Model


NameError: name 'baseline_results' is not defined

: 

# 🔄 Independent Inference Section

**This section can be run independently after fine-tuning is complete!**

Run these cells to:
1. Load your saved fine-tuned model from local storage
2. Load the saved dataset splits
3. Evaluate the model on all 4 splits
4. Compare results with baseline (if available)

**Note:** Make sure you have already trained and saved the model (Section 14)

## Step 1: Configure Paths

Set the paths to your saved model and datasets.

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 1: Configure Paths
Run this cell to set up paths to your saved model and datasets
"""

import os

# Configuration - MODIFY THESE TO MATCH YOUR SAVED MODEL
BASE_MODEL_NAME = "facebook/opt-1.3b"  # Original base model
DATASET_NAME = "sst2"  # Dataset used for training
MODEL_CHECKPOINT_DIR = f'qlora_checkpoints/{BASE_MODEL_NAME.split("/")[-1].lower()}-qlora-{DATASET_NAME.lower()}'

# Paths
SAVED_MODEL_PATH = os.path.join(MODEL_CHECKPOINT_DIR, 'final_model')
DATASETS_PATH = os.path.join(MODEL_CHECKPOINT_DIR, 'datasets')
BASELINE_PATH = os.path.join(MODEL_CHECKPOINT_DIR, 'evaluation_comparison.pkl')

print("="*60)
print("🔧 Configuration")
print("="*60)
print(f"Base Model: {BASE_MODEL_NAME}")
print(f"Dataset: {DATASET_NAME}")
print(f"Model Path: {SAVED_MODEL_PATH}")
print(f"Datasets Path: {DATASETS_PATH}")
print(f"Baseline Results: {BASELINE_PATH}")
print("="*60)

# Verify paths exist
if not os.path.exists(SAVED_MODEL_PATH):
    print(f"\n⚠️  WARNING: Model path not found: {SAVED_MODEL_PATH}")
    print("   Make sure you have trained and saved the model first!")
else:
    print(f"\n✅ Model found at: {SAVED_MODEL_PATH}")

if not os.path.exists(DATASETS_PATH):
    print(f"\n⚠️  WARNING: Datasets path not found: {DATASETS_PATH}")
    print("   You may need to load from HuggingFace instead.")
else:
    print(f"✅ Datasets found at: {DATASETS_PATH}")

## Step 2: Load Fine-tuned Model

Load the saved fine-tuned model with LoRA adapters.

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 2: Load Fine-tuned Model
This cell loads your saved fine-tuned model from local storage
"""

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

print("="*60)
print("🔄 Loading Fine-tuned Model")
print("="*60)

# Load the base model with 4-bit quantization
print("\n1️⃣ Loading base model with 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)
print("   ✅ Base model loaded")

# Load tokenizer
print("\n2️⃣ Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(SAVED_MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("   ✅ Tokenizer loaded")

# Load the fine-tuned LoRA adapters
print("\n3️⃣ Loading fine-tuned LoRA adapters...")
model = PeftModel.from_pretrained(base_model, SAVED_MODEL_PATH)
print("   ✅ LoRA adapters loaded")

# Move to evaluation mode
model.eval()

print("\n" + "="*60)
print("✅ Model loaded successfully!")
print(f"   Device: {model.device}")
print(f"   Model type: {type(model).__name__}")
print("="*60)

## Step 3: Load Dataset Splits

Load the saved dataset splits from local storage (or from HuggingFace if not available locally).

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 3: Load Dataset Splits
Load datasets from local storage or HuggingFace
"""

from datasets import load_from_disk, load_dataset

print("="*60)
print("📊 Loading Dataset Splits")
print("="*60)

dataset = {}

# Try to load from local storage first
if os.path.exists(DATASETS_PATH):
    print("\n🔹 Loading from local storage...")
    for split_name in ['train_retain', 'train_forget', 'test_retain', 'test_forget']:
        split_path = os.path.join(DATASETS_PATH, split_name)
        if os.path.exists(split_path):
            dataset[split_name] = load_from_disk(split_path)
            print(f"   ✅ {split_name}: {len(dataset[split_name])} examples")
        else:
            print(f"   ⚠️  {split_name} not found at {split_path}")
else:
    # Load from HuggingFace if local not available
    print("\n🔹 Loading from HuggingFace...")
    data_path = "karuna-bhaila/Unlearning_SST2v3"
    
    def preprocess_sentiment(examples):
        prompt = f"""### Text: {examples['text']}\n\n### Question: What is the sentiment of the given text?\n\n### Sentiment:"""
        completion = f""" {examples['label_text']}"""
        return {"text": prompt + completion}
    
    raw_dataset = load_dataset(data_path)
    dataset = raw_dataset.map(preprocess_sentiment, batched=False)
    dataset = dataset.remove_columns(['label', 'label_text'])
    dataset.set_format("torch")
    
    for split_name in dataset.keys():
        print(f"   ✅ {split_name}: {len(dataset[split_name])} examples")

print("\n" + "="*60)
print("✅ Datasets loaded successfully!")
print("="*60)

## Step 4: Define Evaluation Function

This function evaluates the model on a given dataset split.

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 4: Define Evaluation Function
"""

from tqdm import tqdm
import torch

def evaluate_model(model, dataset_split, tokenizer, batch_size=8, split_name=""):
    """
    Evaluate model using single-token prediction approach.
    Returns accuracy by checking if the model predicts the correct sentiment token.
    """
    model.eval()
    
    correct = 0
    total = 0
    
    # Map sentiment labels to expected tokens
    sentiment_mapping = {
        'positive': tokenizer.encode(' positive', add_special_tokens=False)[0],
        'negative': tokenizer.encode(' negative', add_special_tokens=False)[0],
        'Positive': tokenizer.encode(' Positive', add_special_tokens=False)[0],
        'Negative': tokenizer.encode(' Negative', add_special_tokens=False)[0],
    }
    
    with torch.no_grad():
        for i in tqdm(range(0, len(dataset_split), batch_size), desc=f"Evaluating {split_name}"):
            batch_end = min(i + batch_size, len(dataset_split))
            batch = dataset_split[i:batch_end]
            
            for example in batch['text']:
                # Split into prompt and completion
                if "### Sentiment:" in example:
                    prompt_part = example.split("### Sentiment:")[0] + "### Sentiment:"
                    true_sentiment = example.split("### Sentiment:")[1].strip()
                    
                    # Tokenize prompt
                    inputs = tokenizer(prompt_part, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {k: v.to(model.device) for k, v in inputs.items()}
                    
                    # Get model prediction
                    outputs = model(**inputs)
                    logits = outputs.logits
                    next_token_logits = logits[0, -1, :]
                    predicted_token = torch.argmax(next_token_logits).item()
                    
                    # Check if predicted token matches any sentiment token
                    true_sentiment_lower = true_sentiment.lower().strip()
                    
                    # Get expected token(s)
                    expected_tokens = []
                    if 'positive' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('positive'), sentiment_mapping.get('Positive')]
                    elif 'negative' in true_sentiment_lower:
                        expected_tokens = [sentiment_mapping.get('negative'), sentiment_mapping.get('Negative')]
                    
                    expected_tokens = [t for t in expected_tokens if t is not None]
                    
                    if predicted_token in expected_tokens:
                        correct += 1
                    
                    total += 1
    
    accuracy = correct / total if total > 0 else 0.0
    return {'accuracy': accuracy, 'correct': correct, 'total': total}

print("✅ Evaluation function defined!")

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 7: Test with Custom Examples
Try your own text samples
"""

def predict_sentiment(text, model, tokenizer):
    """
    Predict sentiment for a given text.
    
    Args:
        text: Input text to analyze
        model: Fine-tuned model
        tokenizer: Tokenizer
        
    Returns:
        predicted_sentiment: The predicted sentiment label
    """
    # Format prompt exactly as during training
    prompt = f"""### Text: {text}\n\n### Question: What is the sentiment of the given text?\n\n### Sentiment:"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Get prediction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        next_token_logits = logits[0, -1, :]
        predicted_token = torch.argmax(next_token_logits).item()
    
    # Decode token
    predicted_text = tokenizer.decode([predicted_token])
    
    # Map to sentiment
    if 'positive' in predicted_text.lower():
        return 'positive', predicted_text
    elif 'negative' in predicted_text.lower():
        return 'negative', predicted_text
    else:
        return 'unknown', predicted_text

# Test examples
test_examples = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Terrible experience. Would not recommend to anyone.",
    "It was okay, nothing special but not bad either.",
    "Best purchase I've ever made! Highly recommended!",
    "Waste of money. Very disappointed with the quality.",
]

print("="*60)
print("🎯 Testing with Custom Examples")
print("="*60)

for i, text in enumerate(test_examples, 1):
    sentiment, token = predict_sentiment(text, model, tokenizer)
    print(f"\nExample {i}:")
    print(f"Text: {text}")
    print(f"Predicted: {sentiment} (token: {token!r})")
    print("-"*60)

print("\n✅ Custom testing complete!")

# Try your own text
print("\n" + "="*60)
print("💡 To try your own text, modify and run:")
print("="*60)
print('your_text = "Your custom text here"')
print('sentiment, token = predict_sentiment(your_text, model, tokenizer)')
print('print(f"Predicted: {sentiment} (token: {token!r})")')
print("="*60)

## Step 7: Test with Custom Examples (Optional)

Try the model with your own text samples.

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 6: Compare with Baseline
Load and compare with baseline results if available
"""

import pickle

# Try to load baseline results
baseline_results = None
if os.path.exists(BASELINE_PATH):
    try:
        with open(BASELINE_PATH, 'rb') as f:
            comparison_data = pickle.load(f)
            baseline_results = comparison_data.get('baseline', None)
        print("✅ Baseline results loaded successfully!")
    except Exception as e:
        print(f"⚠️  Could not load baseline results: {e}")
else:
    print("⚠️  No baseline results found. Skipping comparison.")

# Compare if baseline available
if baseline_results:
    print("\n" + "="*60)
    print("📈 Comparison: Base vs Fine-tuned Model")
    print("="*60)
    
    def compare_split(split_name, baseline, finetuned):
        """Compare baseline and fine-tuned results for a split."""
        print(f"\n🔹 {split_name} Comparison:")
        
        base_acc = baseline.get('accuracy', 0.0)
        ft_acc = finetuned.get('accuracy', 0.0)
        diff = ft_acc - base_acc
        diff_pct = (diff / base_acc * 100) if base_acc > 0 else float('inf')
        arrow = "↑" if diff > 0 else "↓" if diff < 0 else "→"
        
        print(f"   Base Model:      {base_acc:.4f}")
        print(f"   Fine-tuned:      {ft_acc:.4f}")
        print(f"   Improvement:     {diff:+.4f} ({diff_pct:+.2f}%) {arrow}")
    
    compare_split("Train Retain", baseline_results['train_retain'], results['train_retain'])
    compare_split("Train Forget", baseline_results['train_forget'], results['train_forget'])
    compare_split("Test Retain", baseline_results['test_retain'], results['test_retain'])
    compare_split("Test Forget", baseline_results['test_forget'], results['test_forget'])
    
    print("\n" + "="*60)
    
    # Calculate average improvement
    avg_base = sum(baseline_results[k]['accuracy'] for k in baseline_results.keys()) / len(baseline_results)
    avg_ft = sum(results[k]['accuracy'] for k in results.keys()) / len(results)
    avg_improvement = avg_ft - avg_base
    
    print(f"\n📊 Overall Statistics:")
    print(f"   Average Base Accuracy:       {avg_base:.4f}")
    print(f"   Average Fine-tuned Accuracy: {avg_ft:.4f}")
    print(f"   Average Improvement:         {avg_improvement:+.4f} ({(avg_improvement/avg_base*100):+.2f}%)")
    print("="*60)
else:
    print("\n💡 Tip: Run the base model evaluation (Section 8.7) before training to generate baseline results for comparison.")

# Save current results
save_path = os.path.join(MODEL_CHECKPOINT_DIR, 'inference_results.pkl')
with open(save_path, 'wb') as f:
    pickle.dump({
        'results': results,
        'baseline': baseline_results,
        'model_path': SAVED_MODEL_PATH,
    }, f)

print(f"\n✅ Results saved to: {save_path}")

## Step 6: Compare with Baseline (Optional)

If baseline results are available, compare the fine-tuned model with the base model.

In [ ]:
"""
INDEPENDENT INFERENCE - STEP 5: Evaluate on All Splits
Run evaluation on all 4 dataset splits
"""

print("="*60)
print("📊 Fine-tuned Model Evaluation Results")
print("="*60)

# Evaluate on all 4 splits
results = {}

print("\n🔹 Evaluating Train Retain...")
results['train_retain'] = evaluate_model(model, dataset['train_retain'], tokenizer, batch_size=8, split_name="Train Retain")
print(f"   Accuracy: {results['train_retain']['accuracy']:.4f} ({results['train_retain']['correct']}/{results['train_retain']['total']})")

print("\n🔹 Evaluating Train Forget...")
results['train_forget'] = evaluate_model(model, dataset['train_forget'], tokenizer, batch_size=8, split_name="Train Forget")
print(f"   Accuracy: {results['train_forget']['accuracy']:.4f} ({results['train_forget']['correct']}/{results['train_forget']['total']})")

print("\n🔹 Evaluating Test Retain...")
results['test_retain'] = evaluate_model(model, dataset['test_retain'], tokenizer, batch_size=8, split_name="Test Retain")
print(f"   Accuracy: {results['test_retain']['accuracy']:.4f} ({results['test_retain']['correct']}/{results['test_retain']['total']})")

print("\n🔹 Evaluating Test Forget...")
results['test_forget'] = evaluate_model(model, dataset['test_forget'], tokenizer, batch_size=8, split_name="Test Forget")
print(f"   Accuracy: {results['test_forget']['accuracy']:.4f} ({results['test_forget']['correct']}/{results['test_forget']['total']})")

print("\n" + "="*60)
print("✅ Evaluation Complete!")
print("="*60)

## Notes and Tips

### Important: Modern TRL Approach (TRL 0.8.6+)
This notebook uses the **modern TRL approach** with:
- `SFTConfig` instead of `TrainingArguments`
- `completion_only_loss=True` to mask prompt tokens
- No need for `DataCollatorForCompletionOnlyLM` (deprecated/removed)

This ensures only the sentiment label tokens contribute to the loss, not the prompt text.

### Evaluation Strategy
The notebook now includes:
1. **Base Model Evaluation**: Evaluates the model before fine-tuning to establish a baseline
2. **Fine-tuned Model Evaluation**: Evaluates after training on held-out test sets
3. **Comparison Metrics**: Shows improvement (or degradation) across accuracy, F1-score, precision, and recall
4. **Independent Inference**: Load model from Google Drive for standalone inference

### GPU Requirements:
- **Llama-2-7B**: Requires at least 12GB GPU memory (use Colab Pro with A100)
- **OPT-1.3B**: Can run on free Colab with T4 GPU
- **Llama-2-13B**: Requires 16GB+ GPU memory

### Memory Optimization:
- Reduce `train_batch_size` if you run out of memory
- Reduce `max_length` to 512 or 256 for smaller memory footprint
- Use gradient accumulation by adding `gradient_accumulation_steps=4` to `SFTConfig`

### Training Tips:
- Start with fewer epochs (3-5) for initial experimentation
- Monitor the training loss and adjust learning rate if needed
- Use W&B to track experiments and compare different configurations
- The evaluation comparison shows if fine-tuning improved the model

### Inference Tips:
- The inference section (16) is completely independent
- You can run it in a new notebook session by copying just those cells
- Model is saved in Google Drive for persistence across sessions
- Adjust `max_new_tokens` and `temperature` for different generation behaviors

### TRL Version Compatibility:
- This notebook is designed for TRL 0.8.6+
- If using older TRL versions (<0.8), you may need to use `DataCollatorForCompletionOnlyLM`
- For newest TRL versions (0.19+), ensure `completion_only_loss` is available in `SFTConfig`